In [1]:
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
import cv2
import glob
import time
from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler
from skimage.feature import hog
from sklearn.model_selection import train_test_split
from skimage.feature import hog
get_ipython().magic('matplotlib inline')
from scipy.ndimage.measurements import label
import findLaneLines
findLaneLines.initialize()

In [2]:
# Define a function to compute binned color features  
def bin_spatial(img, size=(16, 16)):
    return cv2.resize(img, size).ravel() 

# Define a function to compute color histogram features 
def color_hist(imgch1,imgch2,imgch3, nbins=32):
    ch1 = np.histogram(imgch1, bins=nbins, range=(0, 256))[0]#We need only the histogram, no bins edges
    ch2 = np.histogram(imgch2, bins=nbins, range=(0, 256))[0]
    ch3 = np.histogram(imgch3, bins=nbins, range=(0, 256))[0]
    hist = np.hstack((ch1, ch2, ch3))
    return hist
# Define a function to compute HOG features 
def get_hog_features(img, orient, pix_per_cell, cell_per_block, vis=False, feature_vec=True):
    if vis == True: # Call with two outputs if vis==True to visualize the HOG
        features, hog_image = hog(img, orientations=orient, 
                                  pixels_per_cell=(pix_per_cell, pix_per_cell),
                                  cells_per_block=(cell_per_block, cell_per_block), 
                                  transform_sqrt=True, 
                                  visualise=vis, feature_vector=feature_vec)
        return features, hog_image
    else:      # Otherwise call with one output
        features = hog(img, orientations=orient, 
                       pixels_per_cell=(pix_per_cell, pix_per_cell),
                       cells_per_block=(cell_per_block, cell_per_block), 
                       transform_sqrt=True, 
                       visualise=vis, feature_vector=feature_vec)
        return features
# Define a function to extract features from a list of images
def img_features(feature_image, spatial_feat, hist_feat, hog_feat, hist_bins, orient, 
                        pix_per_cell, cell_per_block, hog_channel, spatial_size=(32, 32)):
    #print(spatial_feat, hist_feat, hog_feat, hist_bins, orient, 
                        #pix_per_cell, cell_per_block, hog_channel, spatial_size)
    
    file_features = []
    feature_image_luv = cv2.cvtColor(feature_image, cv2.COLOR_RGB2LUV)
    feature_image_hls = cv2.cvtColor(feature_image, cv2.COLOR_RGB2HLS)
    feature_image_hsv = cv2.cvtColor(feature_image, cv2.COLOR_RGB2HSV)
    feature_image_gray = cv2.cvtColor(feature_image, cv2.COLOR_RGB2GRAY)
    if spatial_feat == True:
        spatial_features = bin_spatial(feature_image, size=spatial_size)
        #print 'spat', spatial_features.shape
        file_features.append(spatial_features)
    if hist_feat == True:
         # Apply color_hist()
        hist_features = color_hist(feature_image_luv[0],feature_image_hls[2],feature_image_hsv[2], nbins=hist_bins)
        file_features.append(hist_features)
    
    if hog_feat == True:
    # Call get_hog_features() with vis=False, feature_vec=True
        if hog_channel == 'ALL':
            hog_features = []
            for channel in range(feature_image_luv.shape[2]):
                hog_features.append(get_hog_features(feature_image[:,:,channel], 
                                        orient, pix_per_cell, cell_per_block, 
                                        vis=False, feature_vec=True))
                hog_features = np.ravel(hog_features)        
        else:
            hog_features = get_hog_features(feature_image_gray[:,:], orient, 
                            pix_per_cell, cell_per_block, vis=False, feature_vec=True)
            # Append the new feature vector to the features list
        file_features.append(hog_features)
    return np.concatenate(file_features)
def extract_features(imgs, color_space='RGB', spatial_size=(32, 32),
                        hist_bins=32, orient=9, 
                        pix_per_cell=8, cell_per_block=2, hog_channel=0,
                        spatial_feat=True, hist_feat=True, hog_feat=True):
    # Create a list to append feature vectors to
    features = []
    # Iterate through the list of images
    for file_p in imgs:
        file_features = []
        feature_image = cv2.imread(file_p) # Read in each imageone by one
        file_features = img_features(feature_image, spatial_feat, hist_feat, hog_feat, hist_bins, orient, 
                        pix_per_cell, cell_per_block, hog_channel)
        features.append(file_features)
        feature_image=cv2.flip(feature_image,1) # Augment the dataset with flipped images
        file_features = img_features(feature_image, spatial_feat, hist_feat, hog_feat, hist_bins, orient, 
                        pix_per_cell, cell_per_block, hog_channel)
        features.append(file_features)
    return features # Return list of feature vectors
def slide_window(img, x_start_stop=[None, None], y_start_stop=[None, None], 
                    xy_window=(64, 64), xy_overlap=(0.5, 0.5)):
    # If x and/or y start/stop positions not defined, set to image size
    if x_start_stop[0] == None:
        x_start_stop[0] = 0
    if x_start_stop[1] == None:
        x_start_stop[1] = img.shape[1]
    if y_start_stop[0] == None:
        y_start_stop[0] = 0
    if y_start_stop[1] == None:
        y_start_stop[1] = img.shape[0]
    # Compute the span of the region to be searched    
    xspan = x_start_stop[1] - x_start_stop[0]
    yspan = y_start_stop[1] - y_start_stop[0]
    # Compute the number of pixels per step in x/y
    nx_pix_per_step = np.int(xy_window[0]*(1 - xy_overlap[0]))
    ny_pix_per_step = np.int(xy_window[1]*(1 - xy_overlap[1]))
    # Compute the number of windows in x/y
    nx_buffer = np.int(xy_window[0]*(xy_overlap[0]))
    ny_buffer = np.int(xy_window[1]*(xy_overlap[1]))
    nx_windows = np.int((xspan-nx_buffer)/nx_pix_per_step) 
    ny_windows = np.int((yspan-ny_buffer)/ny_pix_per_step) 
    # Initialize a list to append window positions to
    window_list = []
    # Loop through finding x and y window positions
    # Note: you could vectorize this step, but in practice
    # you'll be considering windows one by one with your
    # classifier, so looping makes sense
    for ys in range(ny_windows):
        for xs in range(nx_windows):
            # Calculate window position
            startx = xs*nx_pix_per_step + x_start_stop[0]
            endx = startx + xy_window[0]
            starty = ys*ny_pix_per_step + y_start_stop[0]
            endy = starty + xy_window[1]
            # Append window position to list
            window_list.append(((startx, starty), (endx, endy)))
    # Return the list of windows
    return window_list

# Define a function to draw bounding boxes on an image
def draw_boxes(img, bboxes, color=(0, 0, 255), thick=6):
    imcopy = np.copy(img) # Make a copy of the image
    for bbox in bboxes: # Iterate through the bounding boxes
        cv2.rectangle(imcopy, bbox[0], bbox[1], color, thick)
    return imcopy
# Define a function you will pass an image 
# and the list of windows to be searched (output of slide_windows())
def search_windows(img, windows, clf, scaler, color_space='RGB', 
                    spatial_size=(32, 32), hist_bins=32, 
                    hist_range=(0, 256), orient=8, 
                    pix_per_cell=8, cell_per_block=2, 
                    hog_channel=0, spatial_feat=True, 
                    hist_feat=True, hog_feat=True):

    #1) Create an empty list to receive positive detection windows
    on_windows = []
    #2) Iterate over all windows in the list
    for window in windows:
        #3) Extract the test window from original image
        test_img = cv2.resize(img[window[0][1]:window[1][1], window[0][0]:window[1][0]], (64, 64))
        #print("YYYYY")
        #4) Extract features for that window using single_img_features()
        #print('Pass-', spatial_size)
        features = img_features(test_img, 
                            spatial_size=spatial_size,spatial_feat=spatial_feat,hist_feat=hist_feat, 
                            hog_feat=hog_feat, hist_bins=hist_bins, 
                            orient=orient, pix_per_cell=pix_per_cell, 
                            cell_per_block=cell_per_block, 
                            hog_channel=hog_channel)
        #5) Scale extracted features to be fed to classifier
        #print(features.shape)
        test_features = scaler.transform(np.array(features).reshape(1, -1))
        #6) Predict using your classifier
        prediction = clf.predict(test_features)
        #7) If positive (prediction == 1) then save the window
        if prediction == 1:
            on_windows.append(window)
    #8) Return windows for positive detections
    return on_windows

# A function to show an image
def show_img(img):
    if len(img.shape)==3: #Color BGR image
        plt.figure()
        plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    else: # Grayscale image
        plt.figure()
        plt.imshow(img, cmap='gray')
def add_heat(heatmap, bbox_list):
    # Iterate through list of bboxes
    for box in bbox_list:
        # Add += 1 for all pixels inside each bbox
        # Assuming each "box" takes the form ((x1, y1), (x2, y2))
        heatmap[box[0][1]:box[1][1], box[0][0]:box[1][0]] += 1
    return heatmap # Return updated heatmap
    
def apply_threshold(heatmap, threshold): # Zero out pixels below the threshold in the heatmap
    heatmap[heatmap < threshold] = 0 
    return heatmap 

def draw_labeled_bboxes1(img, labels):
    # Iterate through all detected cars
    for car_number in range(1, labels[1]+1):
        # Find pixels with each car_number label value
        nonzero = (labels[0] == car_number).nonzero()
        # Identify x and y values of those pixels
        nonzeroy = np.array(nonzero[0])
        nonzerox = np.array(nonzero[1])
        # Define a bounding box based on min/max x and y
        bbox = ((np.min(nonzerox), np.min(nonzeroy)), (np.max(nonzerox), np.max(nonzeroy)))
        # Draw the box on the image
        if (((np.max(nonzerox)-np.min(nonzerox))>50) and ((np.max(nonzeroy)-np.min(nonzeroy))>50)):
            cv2.rectangle(img, bbox[0], bbox[1], (0,0,255), 6)
    # Return the image
    return img
def find_labeled_bboxes(labels):
    # Iterate through all detected cars
    bbox_list=[]
    for car_number in range(1, labels[1]+1):
        # Find pixels with each car_number label value
        nonzero = (labels[0] == car_number).nonzero()
        # Identify x and y values of those pixels
        nonzeroy = np.array(nonzero[0])
        nonzerox = np.array(nonzero[1])
        # Define a bounding box based on min/max x and y
        bbox_list.append(((np.min(nonzerox), np.min(nonzeroy)), (np.max(nonzerox), np.max(nonzeroy))))
        # Draw the box on the image
        #if (((np.max(nonzerox)-np.min(nonzerox))>50) and ((np.max(nonzeroy)-np.min(nonzeroy))>50)):
            #cv2.rectangle(img, bbox[0], bbox[1], (0,0,255), 6)
    # Return the car boxes
    return bbox_list
def print_boxes(img,boxes):
    for car_number in range(len(boxes)):
        # Draw the box on the image
        if (((boxes[car_number][1][1]-boxes[car_number][0][1])>60) and ((boxes[car_number][1][0]-boxes[car_number][0][0])>60)):
            img=cv2.rectangle(img, boxes[car_number][0], boxes[car_number][1], (0,0,255), 6)    
            #print('d1-',boxes[car_number][0],'d2-',boxes[car_number][1])
    return img

In [3]:
images = glob.glob('*vehicles/*/*')
cars = []
notcars = []
for image in images:
    if 'non' in image:
        notcars.append(image)
    else:
        cars.append(image)
print(len(cars))
print(len(notcars))


8792
8968


In [4]:
color_space = 'RGB' # Can be RGB, HSV, LUV, HLS, YUV, YCrCb
orient = 8  # HOG orientations
pix_per_cell = 8 # HOG pixels per cell
cell_per_block = 2 # HOG cells per block
hog_channel = 0 # Can be 0, 1, 2, or "ALL"
spatial_size = (32, 32) # Spatial binning dimensions
hist_bins = 32    # Number of histogram bins
spatial_feat = True # Spatial features on or off
hist_feat = True # Histogram features on or off
hog_feat = True # HOG features on or off

car_features = extract_features(cars, color_space=color_space, 
                        spatial_size=spatial_size, hist_bins=hist_bins, 
                        orient=orient, pix_per_cell=pix_per_cell, 
                        cell_per_block=cell_per_block, 
                        hog_channel=hog_channel, spatial_feat=spatial_feat, 
                        hist_feat=hist_feat, hog_feat=hog_feat)
print ('Car samples: ', len(car_features))
notcar_features = extract_features(notcars, color_space=color_space, 
                        spatial_size=spatial_size, hist_bins=hist_bins, 
                        orient=orient, pix_per_cell=pix_per_cell, 
                        cell_per_block=cell_per_block, 
                        hog_channel=hog_channel, spatial_feat=spatial_feat, 
                        hist_feat=hist_feat, hog_feat=hog_feat)
print ('Notcar samples: ', len(notcar_features))
X = np.vstack((car_features, notcar_features)).astype(np.float64)                        

X_scaler = StandardScaler().fit(X) # Fit a per-column scaler
scaled_X = X_scaler.transform(X) # Apply the scaler to X
print(scaled_X.shape)

y = np.hstack((np.ones(len(car_features)), np.zeros(len(notcar_features)))) # Define the labels vector

# Split up data into randomized training and test sets
X_train, X_test, y_train, y_test = train_test_split(scaled_X, y, test_size=0.2, random_state=22)

print('Using:',orient,'orientations', pix_per_cell,
    'pixels per cell and', cell_per_block,'cells per block')
print('Feature vector length:', len(X_train[0]))
svc = LinearSVC(loss='hinge') # Use a linear SVC 
t=time.time() # Check the training time for the SVC
svc.fit(X_train, y_train) # Train the classifier
t2 = time.time()
print(round(t2-t, 2), 'Seconds to train SVC...')
print('Test Accuracy of SVC = ', round(svc.score(X_test, y_test), 4)) # Check the score of the SVC


Car samples:  17584
Notcar samples:  17936
(35520, 4736)
Using: 8 orientations 8 pixels per cell and 2 cells per block
Feature vector length: 4736
23.35 Seconds to train SVC...
Test Accuracy of SVC =  0.9837


In [5]:
def find_vehicles(image):
    t=time.time()
    prev_cars = []
    windows = []
    number_layers = 2
    global car_boxes
    global frame
    global heat_prime
    global all_car_boxes
    global possible_car_boxes
    frame +=1
    image=cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    search_layers = [[(700,1280),(400,670),(100,100),(0.75,0.75)],
                     [(700,1280),(400,590),(64,64),(0.75,0.75)],
                     [(750,1000),(400,510),(30,30),(0.75,0.75)]]
    #heat_prime = np.zeros((720,1280))
    draw_image = np.copy(image)
    if (frame%6 == 1):
        possible_car_boxes = []
    if (frame%3 == 1):
        for n in range(number_layers):
            windows += slide_window(image, search_layers[n][0],search_layers[n][1],search_layers[n][2],search_layers[n][3])
    
        if (len(possible_car_boxes) > 0):
            for i in possible_car_boxes:
                search_win = (int((650-i[1][1])/3),int((650-i[1][1])/3))
                windows += slide_window(image,(i[0][0]-5,i[1][0]),(i[0][1],i[1][1]),search_win,(0.75,0.75))
    hot_windows = []
    hot_windows += (search_windows(image, windows, svc, X_scaler, color_space=color_space, 
                        spatial_size=(32,32), hist_bins=hist_bins, 
                        orient=orient, pix_per_cell=pix_per_cell, 
                        cell_per_block=cell_per_block, 
                        hog_channel=hog_channel, spatial_feat=spatial_feat, 
                        hist_feat=hist_feat, hog_feat=hog_feat))  
    if (len(hot_windows) > 2):
        heat = np.zeros_like(image[:,:,0]).astype(np.float)
        heat = add_heat(heat, hot_windows)
        heat_prime = heat_prime + heat
        heat_prime = apply_threshold(heat_prime,3) # Apply threshold to help remove false positives
        # Visualize the heatmap when displaying    
        poss_labels = label(heat_prime)
        possible_car_boxes.extend(find_labeled_bboxes(poss_labels))
        
    if (frame%6 == 1):
        
        heat_prime = apply_threshold(heat_prime,5)
        labels = label(heat_prime)
        heat_prime = np.zeros((720,1280))
        if (labels[1] > 0):
            all_car_boxes=[]
            
            all_car_boxes.extend(find_labeled_bboxes(labels))
            
    if (len(all_car_boxes) > 0):
        image = print_boxes(image,all_car_boxes)
        
    return findLaneLines.lane_detection(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

In [8]:
from moviepy.editor import VideoFileClip
frame = 0
car_boxes = []
heat_prime = np.zeros((720,1280))
all_car_boxes = []
possible_car_boxes=[]

output_v = 'project_video_output_final.mp4'
clip1 = VideoFileClip("project_video.mp4")
clip = clip1.fl_image(find_vehicles)
get_ipython().magic('time clip.write_videofile(output_v, audio=False)')

[MoviePy] >>>> Building video project_video_output_final.mp4
[MoviePy] Writing video project_video_output_final.mp4


100%|█████████▉| 1260/1261 [13:34<00:00,  1.37it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: project_video_output_final.mp4 

CPU times: user 18min 28s, sys: 26.2 s, total: 18min 54s
Wall time: 13min 34s
